In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark import StorageLevel
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis import Extent
from geopyspark.geotrellis.rdd import TiledRasterRDD, Pyramid
from geopyspark.geotrellis.color import get_breaks, ColorMap
from geopyspark.geotrellis.constants import *
import json
import shapely
from shapely.geometry import shape, asShape, MultiPoint, MultiLineString
import matplotlib.pyplot as plt

from geonotebook.wrappers import RddRasterData

%matplotlib inline

conf = SparkConf()
conf.setMaster("local[*]")

gps = GeoPyContext(SparkContext.getOrCreate(conf))

## Rasterize Libya Roads to Raster RDD

In [ ]:
with open('roads.geojson', encoding='utf-8') as data_file:
    roads = json.load(data_file)
    libya_roads = MultiLineString([shape(geom['geometry']) for geom in roads['features']])

with open('Libya_shape.geojson', encoding='utf-8') as data_file:
    libya_border = shape(json.load(data_file)['features'][0]['geometry'])

In [ ]:
# Try to be friedly with python libraries like shapely
from geopyspark import geotrellis
Extent(*libya_border.bounds)
Extent(*libya_roads.bounds)

In [ ]:
# All execution time here is sending WKB over py4j socket
from geopyspark.geotrellis.rdd import rasterize
from geopyspark.geotrellis import RasterizerOptions

ro = RasterizerOptions(includePartial=True, sampleType='PixelIsArea')

road_raster = rasterize(gps, geoms=libya_roads.geoms, 
                        crs="EPSG:3857", zoom=8, 
                        fill_value=1, cell_type="float32",
                        options = ro,
                        numPartitions = 20)

road_raster.layer_metadata.bounds

## Show Rasterized Roads on a Map

In [ ]:
# Pyramid up from base layer

road_pp = road_raster.pyramid(start_zoom=8, end_zoom=1).cache()

In [ ]:
from geopyspark.geotrellis.tms import *
from geonotebook.wrappers.raster import TMSRasterData

# color map roads 1 to red
roads_cm = ColorMap.from_colors(gps, [1], [0xff000080])

# start JVM tile server and serve tiles to map
server = rdd_tms_server(gps, road_pp, roads_cm)
M.add_layer(TMSRasterData(server), name="TMS")

In [ ]:
M.remove_layer(M.layers[0])

## Cost Distance Based on Road Network

In [ ]:
# road network will shape our friction layer

road_friction = (
    road_raster
    .reclassify({1:1}, data_type=int, replace_nodata_with=10)
)

In [ ]:
# starting points for cost distance operation

with open('population.geojson', encoding='utf-8') as data_file:
    population_centers = MultiPoint([shape(geom['geometry']) for geom in json.load(data_file)['features']])

with open('conflict.geojson', encoding='utf-8') as data_file:
    conflict_centers=MultiPoint([shape(feature['geometry']) for feature in json.load(data_file)['features'] if feature['geometry'] != None])

conflict_centers

In [ ]:
from geonotebook.wrappers import VectorData
M.add_layer(VectorData("population.geojson"), name="Population")

In [ ]:
# wait for the iterations to settle

pop_cd = (
    road_friction
    .cost_distance(
        geometries = population_centers, 
        max_distance=1400000.0
    )
)
pop_pp = pop_cd.pyramid(start_zoom=8, end_zoom=1)

In [ ]:
con_cd = (
    road_friction
    .cost_distance(
        geometries = conflict_centers, 
        max_distance=1400000.0
    )
)
con_pp = con_cd.pyramid(start_zoom=8, end_zoom=1)

In [ ]:
# prepare color map for weighted overlay based on max cost

breaks = [x for x in range(0, 1000000, 10000)]
colors = get_breaks(gps, 'Viridis', len(breaks))
wo_cm = ColorMap.from_colors(gps, breaks=breaks, color_list=colors)

In [ ]:
from geopyspark.geotrellis.tms import *
from geonotebook.wrappers.raster import TMSRasterData

In [ ]:
weighted_overlay = (con_pp * 0.0) + (pop_pp * 1.0)

server = rdd_tms_server(gps, weighted_overlay, wo_cm)
M.add_layer(TMSRasterData(server), name="WO")

In [ ]:
# remove the next to last layer
M.remove_layer(M.layers[-2])